In [1]:
import pandas as pd
#from coleta_dados import DadosFundamentalistas
from chat_bots import chat_bot_fundamentalistas, get_secret_key
from coleta_dados import DadosFundamentalistas, DadosNoticiasBuscadorYahoo
from tratando_dados import tratando_dados_fundamentalistas
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
tic = 'BBDC3'
data_inicio = '2020-01-01'

dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)
trat = tratando_dados_fundamentalistas(tic, data_inicio)

In [3]:
dados = dados_fundamentalistas.dados_fundamentalistas_completo()

In [4]:
query = 'Como esta a saude financeira da bbdc3'
response = chat_bot_fundamentalistas(query=query, dados=trat)

In [5]:
response.split('\n\n')

['**Análise da Saúde Financeira da BBDC3**',
 '**Resumo da Análise**',
 'A BBDC3 é uma empresa de investimento que opera no mercado de valores mobiliários. A análise dos indicadores financeiros da empresa mostra uma tendência de crescimento nos resultados financeiros e uma melhoria na gestão da dívida. No entanto, a empresa ainda apresenta alguns riscos e desafios que precisam ser monitorados.',
 '**Pontos Fortes**',
 '* **Crescimento dos Resultados Financeiros**: A BBDC3 apresentou um crescimento significativo nos resultados financeiros nos últimos anos, com um aumento de 15% na receita líquida e de 20% no lucro líquido.\n* **Melhoria na Gestão da Dívida**: A empresa reduziu a dívida líquida em 10% nos últimos anos, o que é um sinal de boa gestão financeira.\n* **Elevada Margem de Lucro**: A BBDC3 apresenta uma margem de lucro líquido de 6,1%, o que é considerado elevado para a indústria de investimento.',
 '**Pontos Fracos**',
 '* **Risco de Dívida**: A empresa ainda apresenta um ris

In [6]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

noticias_yahoo = DadosNoticiasBuscadorYahoo('PETR4', chrome_options)

dados = noticias_yahoo.get_news(1)


Tipo relevancia já está selecionado!


In [8]:
dados

{'links': ['https://valorinveste.globo.com/mercados/renda-variavel/empresas/noticia/2025/01/23/petroleo-cai-com-trump-e-arrasta-acoes-da-petrobras-petr4-prio-prio3-e-brava-brav3.ghtml',
  'https://einvestidor.estadao.com.br/noticias/petrobras-petr4-investir-acoes-dividendos/',
  'https://valorinveste.globo.com/mercados/renda-variavel/empresas/noticia/2025/01/24/vale-vale3-petrobras-petr4-tenda-tend3-veja-os-destaques-das-empresas.ghtml',
  'https://www.infomoney.com.br/mercados/ibovespa-hoje-bolsa-de-valores-ao-vivo-24012025/',
  'https://einvestidor.estadao.com.br/ultimas/ibovespa-hoje-acoes-petrobras-petr3-petr4-magda-chambriard-dividendos/',
  'https://br.investing.com/news/general/santander-revisa-projecao-para-petroleo-e-ve-petrobras-como-top-pick-no-setor-1444354',
  'https://einvestidor.estadao.com.br/ultimas/lucro-irb-irbr3-pode-ser-maior-4-trimestre-estimativas-btg/',
  'https://www.infomoney.com.br/mercados/petrobras-petr4-nao-deve-reajustar-precos-por-enquanto-apontam-analis